In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials

#### Gathering datasets into a single datasource

In [2]:
result = pd.DataFrame()
    
result.head()

""


In [3]:
url_confirmed = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
url_death = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
url_recovered = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
source_confirmed = pd.read_csv(url_confirmed,index_col=0)
source_death  = pd.read_csv(url_death,index_col=0)
source_recovered = pd.read_csv(url_recovered,index_col=0)


#### Transforming column date

In [4]:
source_confirmed = pd.melt(source_confirmed.reset_index(), id_vars=['Province/State','Country/Region','Lat','Long'],var_name ='Date', value_name='Confirmed')
source_death = pd.melt(source_death.reset_index(), id_vars=['Province/State','Country/Region','Lat','Long'],var_name ='Date', value_name='Deaths')
source_recovered = pd.melt(source_recovered.reset_index(), id_vars=['Province/State','Country/Region','Lat','Long'], var_name ='Date', value_name='Recovered')


In [5]:
# Get names of indexes for which column Country/Region has value US
##indexNamesConf = source_confirmed[source_confirmed['Country/Region'] == 'US' ].index
##source_confirmed = source_confirmed.drop(indexNamesConf, axis=0)

# Get names of indexes for which column Country/Region has value US
##indexNamesDeath = source_death[source_death['Country/Region'] == 'US' ].index
##source_death = source_death.drop(indexNamesDeath, axis=0)



In [6]:
source_confirmed[source_confirmed["Country/Region"] == "US"]
source_death[source_death["Country/Region"] == "US"]

,Province/State,Country/Region,Lat,Long,Date,Deaths
242,NaN,US,40.0,-100.0,1/22/20,0
508,NaN,US,40.0,-100.0,1/23/20,0
774,NaN,US,40.0,-100.0,1/24/20,0
1040,NaN,US,40.0,-100.0,1/25/20,0
1306,NaN,US,40.0,-100.0,1/26/20,0
...,...,...,...,...,...,...
53176,NaN,US,40.0,-100.0,8/8/20,162423
53442,NaN,US,40.0,-100.0,8/9/20,162938
53708,NaN,US,40.0,-100.0,8/10/20,163463
53974,NaN,US,40.0,-100.0,8/11/20,164527


In [7]:
result = source_confirmed.merge(source_death, how='outer')
result = result.merge(source_recovered, how='outer')

##result = result.merge(result_US, how='outer')

In [8]:
result["Date"] = pd.to_datetime(result["Date"])

result = result.reset_index()

result.rename(columns={"Country/Region": "Country"}, inplace=True)

In [9]:
##result = result.reset_index()
result.set_index('Country','Date', inplace=True)

In [10]:
#Index:  df.set_index('DateTime1', drop=True, append=False, inplace=True, verify_integrity=False)
#        df = df.sort_index()

# lag confirmed by Dates

##result = result.sort_values(by=["Country/Region"],ascending=True)

##us = result.loc['US']

##us = us.reset_index()

##result = result.reset_index()

##result.set_index('Country/Region', inplace=True)

##us.set_index('Date', inplace=True)

##### Grouping the Data Frame in order to correct some values in regard of Province and Location

In [11]:
result = result.groupby(['Country','Date'])['Confirmed','Deaths','Recovered'].sum()
result.head(1000)

Confirmed  Deaths  Recovered
Country     Date                                    
Afghanistan 2020-01-22        0.0     0.0        0.0
            2020-01-23        0.0     0.0        0.0
            2020-01-24        0.0     0.0        0.0
            2020-01-25        0.0     0.0        0.0
            2020-01-26        0.0     0.0        0.0
...                           ...     ...        ...
Angola      2020-07-19      705.0    29.0      221.0
            2020-07-20      749.0    29.0      221.0
            2020-07-21      779.0    30.0      221.0
            2020-07-22      812.0    33.0      221.0
            2020-07-23      851.0    33.0      236.0

[1000 rows x 3 columns]

In [12]:
result['Confirmed_Lag'] = result.groupby('Country')['Confirmed'].shift(1)
result['Deaths_Lag'] = result.groupby('Country')['Deaths'].shift(1)
result['Recovered_Lag'] = result.groupby('Country')['Recovered'].shift(1)

In [13]:
result['Confirmed_Growth'] = result['Confirmed'] - result['Confirmed_Lag'] 
result['Deaths_Growth']    = result['Deaths']    - result['Deaths_Lag'] 
result['Recovered_Growth'] = result['Recovered'] - result['Recovered_Lag'] 

In [14]:
result.head(10000)

Confirmed  Deaths  Recovered  Confirmed_Lag  \
Country          Date                                                      
Afghanistan      2020-01-22        0.0     0.0        0.0            NaN   
                 2020-01-23        0.0     0.0        0.0            0.0   
                 2020-01-24        0.0     0.0        0.0            0.0   
                 2020-01-25        0.0     0.0        0.0            0.0   
                 2020-01-26        0.0     0.0        0.0            0.0   
...                                ...     ...        ...            ...   
Diamond Princess 2020-08-12      712.0    13.0      651.0          712.0   
Djibouti         2020-01-22        0.0     0.0        0.0            NaN   
                 2020-01-23        0.0     0.0        0.0            0.0   
                 2020-01-24        0.0     0.0        0.0            0.0   
                 2020-01-25        0.0     0.0        0.0            0.0   

                             Deaths_Lag  Recovered_Lag  Confirmed_Growth  \
Country          Date                                                      
Afghanistan      2020-01-22         NaN            NaN               NaN   
                 2020-01-23         0.0            0.0               0.0   
                 2020-01-24         0.0            0.0               0.0   
                 2020-01-25         0.0            0.0               0.0   
                 2020-01-26         0.0            0.0               0.0   
...                                 ...            ...               ...   
Diamond Princess 2020-08-12        13.0          651.0               0.0   
Djibouti         2020-01-22         NaN            NaN               NaN   
                 2020-01-23         0.0            0.0               0.0   
                 2020-01-24         0.0            0.0               0.0   
                 2020-01-25         0.0            0.0               0.0   

                             Deaths_Growth  Recovered_Growth  
Country          Date                                         
Afghanistan      2020-01-22            NaN               NaN  
                 2020-01-23            0.0               0.0  
                 2020-01-24            0.0               0.0  
                 2020-01-25            0.0               0.0  
                 2020-01-26            0.0               0.0  
...                                    ...               ...  
Diamond Princess 2020-08-12            0.0               0.0  
Djibouti         2020-01-22            NaN               NaN  
                 2020-01-23            0.0               0.0  
                 2020-01-24            0.0               0.0  
                 2020-01-25            0.0               0.0  

[10000 rows x 9 columns]

In [15]:
result = result.groupby(['Country','Date'])['Confirmed','Deaths','Recovered','Confirmed_Growth','Deaths_Growth','Recovered_Growth'].sum()

#### First Dates in regard of confirmed, deaths and recovered

In [16]:
first_confirmed = result.query('Confirmed > 0').groupby('Country').head(1)
first_death     = result.query('Deaths > 0').groupby('Country').head(1)
first_recovered = result.query('Recovered > 0').groupby('Country').head(1)

In [17]:
first_confirmed.head()

,,Confirmed,Deaths,Recovered,Confirmed_Growth,Deaths_Growth,Recovered_Growth
Country,Date,,,,,,
Afghanistan,2020-02-24,1.0,0.0,0.0,1.0,0.0,0.0
Albania,2020-03-09,2.0,0.0,0.0,2.0,0.0,0.0
Algeria,2020-02-25,1.0,0.0,0.0,1.0,0.0,0.0
Andorra,2020-03-02,1.0,0.0,0.0,1.0,0.0,0.0
Angola,2020-03-20,1.0,0.0,0.0,1.0,0.0,0.0


In [18]:
first_confirmed = first_confirmed.reset_index()
first_death     = first_death.reset_index()
first_recovered = first_recovered.reset_index()
result          = result.reset_index()

#### First Confirmed Date

In [19]:
date = ''
country = ''
for i, row1 in first_confirmed.iterrows():
    country = row1['Country']
    date = row1['Date']
    
    for i2, row2 in result.iterrows():
        if result.loc[i2, 'Country'] == country:
            result.loc[i2, 'First_Confirmed'] = date

#### First Death Date

In [20]:
for i, row1 in first_death.iterrows():
    country = row1['Country']
    date = row1['Date']
    
    for i2, row2 in result.iterrows():
        if result.loc[i2, 'Country'] == country:
            result.loc[i2, 'First_Death'] = date

#### First Recovered Day

In [ ]:
for i, row1 in first_recovered.iterrows():
    country = row1['Country']
    date = row1['Date']
    
    for i2, row2 in result.iterrows():
        if result.loc[i2, 'Country'] == country:
            result.loc[i2, 'First_Recovered'] = date

In [ ]:
result.head(1000) 

In [ ]:
result.dtypes

In [ ]:
def numOfDays(date1, date2):
    if date2 > date1:
        return 0
    else:
        return abs((date2-date1).days)

#### Calculate days since first cases, deaths and recovered and AVG by day

In [ ]:
for i, row in result.iterrows():
    result.loc[i, 'Days_Since_First_Case']      = numOfDays(result.loc[i, 'Date'], row['First_Confirmed'])
    result.loc[i, 'Days_Since_First_Death']     = numOfDays(result.loc[i, 'Date'], row['First_Death'])
    result.loc[i, 'Days_Since_First_Recovered'] = numOfDays(result.loc[i, 'Date'], row['First_Recovered'])
    result.loc[i, 'Avg_Cases_by_Day']           = result.loc[i, 'Confirmed'] / result.loc[i, 'Days_Since_First_Case']
    result.loc[i, 'Avg_Cases_by_Death']         = result.loc[i, 'Deaths'] / result.loc[i, 'Days_Since_First_Death'] 
    result.loc[i, 'Avg_Cases_by_Recovered']     = result.loc[i, 'Recovered'] / result.loc[i, 'Days_Since_First_Recovered']

In [ ]:
result.set_index('Country','Date', inplace=True)

In [ ]:
br = result.loc['Brazil']

In [ ]:
br.head(1000)

In [ ]:
result.head(1000)

In [ ]:
result = result.to_csv(index=True)

#### Use creds to create a client to interact with the Google Drive API

In [ ]:
SPREADSHEET_ID = '1Y2WhnoWoXII8CsFSJ0b8cypEXgyA0H_TQQJJjjFMpBA'
worksheet_name = 'Covid-19'
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('Covid-19 Reports-452f66ad4e2b.json', scope)
client = gspread.authorize(creds)

#### Find a workbook by name and open the first sheet

In [ ]:
sheet = client.open(worksheet_name).sheet1

### Inserting data to the Google SpreadSheet

In [ ]:
client.import_csv(SPREADSHEET_ID, result)